In [1]:
from IPython.display import display, HTML
from datetime import datetime, timedelta
from lxml import html, etree, objectify
import pandas as pd
import requests
import threading
import queue
import pytz
import math
import csv
import os

In [2]:
def get_symbols():
    with open("n16_symbols.csv", "r") as f:
        symbols = f.read().split()
    return list(set(symbols))
symbols = get_symbols()
len(symbols)

3198

<h3>1. We need to get data from Nasdaq</h3>

In [3]:
def get_text_val(t):
    """
    Helper function
    converts scrapped values into the right numeric representation
    None -> 0
    1,000 -> 1000
    """
    string_value = ("0" if t is None else t).replace(",", "")
    return float(string_value) if string_value.isnumeric() else string_value


get_text_val("1,400")

1400.0

<h3>Let's show where the scrapped values will come from</h3>

<h4>Price</h4>
<img width="600px" align="left" src="price.png">




<h4>Institutional Ownership</h4>
<img width="600px" align="left" src="institutional_ownership.png">



<h4>Active positions</h4>
<img width="600px" align="left" src="active_positions.png">


<h4>New and sold out positions</h4>
<img width="600px" align="left" src="new_and_sold_out_positions.png">

In [4]:
def get_nasdaq_data(symbol):
    """
    Pull some data for a symbol from its nasdaq detail page
    Returns: dict like {total_shares: 1, institutional_ownership: 2, ..}
    """
    url = "http://www.nasdaq.com/symbol/{}/institutional-holdings".format(symbol.lower())
    page = requests.get(url)
    tree = html.fromstring(page.content)    
    
    data = {}
    # Price 
    div = tree.find('.//div[@id="qwidget_lastsale"]')
    if div is not None:
        data["price"] = div.text
    
    # Institutional Ownership
    table = tree.find('.//div[@class="infoTable marginT15px marginL15px"]')
    if table is not None:
        for row in table.findall('.//tr'):
            th_cell, td_cell = row
            title, value = th_cell.text, get_text_val(td_cell.text)
            if title == "Total Shares Outstanding (millions)":
                data["total_shares"] = value
            elif title == "Institutional Ownership":
                data["institutional_ownership"] = value
    
    # Active positions
    table = tree.find('.//div[@class="infoTable paddingT5px"]')
    if table is not None:
        for row in table.findall('.//tr'):
            first, second, third = row
            label, holders, shares = first.text, get_text_val(second.text), get_text_val(third.text)
            if label == "Increased Positions":
                data["increased_holders"] = holders
                data["increased_shares"] = shares
            elif label == "Decreased Positions":
                data["decreased_holders"] = holders
                data["decreased_shares"] = shares
            elif label == "Held Positions":
                data["held_holders"] = holders
                data["held_shares"] = shares
    
    # New and sold out positions    
    table = tree.find('.//div[@class="infoTable floatL marginT15px"]')
    if table is not None:
        for row in table.findall('.//tr'):
            first, second, third = row
            label, holders, shares = first.text, get_text_val(second.text), get_text_val(third.text)
            if label == "New Positions":
                data["new_holders"] = holders
                data["new_shares"] = shares
            elif label == "Sold Out Positions":
                data["sold_holders"] = holders
                data["sold_shares"] = shares
            
    return data

get_nasdaq_data("TSLA")

{'decreased_holders': 322.0,
 'decreased_shares': 8505097.0,
 'held_holders': 153.0,
 'held_shares': 80150177.0,
 'increased_holders': 375.0,
 'increased_shares': 10999115.0,
 'institutional_ownership': '58.69%',
 'new_holders': 109.0,
 'new_shares': 800485.0,
 'price': '$304.34',
 'sold_holders': 92.0,
 'sold_shares': 2277425.0,
 'total_shares': 170.0}

<h3>For the last table we will need some data from yahoo.com</h3>

In [5]:
DATA_TIMEZONE = pytz.timezone("US/Eastern")

def get_the_quarter_days():
    now = datetime.now()
    this_quarter_num = math.ceil(now.month / 3)
    
    if this_quarter_num == 1:
        quarter_num = 4 
        year = now.year - 1
    else:
        quarter_num = this_quarter_num - 1
        year = now.year
        
    start_day = datetime(year, 3 * quarter_num - 2, 1, 9, 30, 0, tzinfo=DATA_TIMEZONE)
    end_day = datetime(year, 3 * this_quarter_num - 2, 1, 13, 30, 0, tzinfo=DATA_TIMEZONE) - timedelta(days=1)
    
    return start_day, end_day
    
get_the_quarter_days()

(datetime.datetime(2018, 1, 1, 9, 30, tzinfo=<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>),
 datetime.datetime(2018, 3, 31, 13, 30, tzinfo=<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>))

In [6]:
def get_data_yahoo(symbol, start, end):
    url = "https://query1.finance.yahoo.com/v7/finance/chart/{}?period1={:.0f}&period2={:.0f}"\
          "&interval=1d&indicators=quote&includeTimestamps=true&"\
          "events=div%7Csplit%7Cearn".format(symbol, start.timestamp(), end.timestamp())
    page = requests.get(url)
    result = page.json()['chart']['result']
    if not result:
        print(symbol, page.json())
        return
    
    data = result[0]
    quote = data['indicators']['quote'][0]
    if 'timestamp' not in data:
        print("Unexpected response for {}".format(symbol))
        return
    
    result = pd.DataFrame(
        index=pd.Index(data=[datetime.fromtimestamp(t).date() for t in data['timestamp']], name="Date"),
        data=dict(
            Open=quote["open"],
            High=quote["high"],
            Low=quote["low"],
            Close=quote["close"],
            Volume=quote["volume"],
        )
    ) 
    return result

start, end = get_the_quarter_days()
yh_df = get_data_yahoo('BAC', start, end)
yh_df

,Close,High,Low,Open,Volume
Date,,,,,
2018-01-02,29.900000,29.900000,29.610001,29.750000,57121600
2018-01-03,29.799999,29.940001,29.690001,29.900000,57865700
2018-01-04,30.190001,30.440001,29.879999,29.969999,76512500
2018-01-05,30.330000,30.420000,30.049999,30.370001,56445200
2018-01-08,30.120001,30.270000,30.049999,30.230000,42914800
2018-01-09,30.270000,30.540001,30.129999,30.200001,69479100
2018-01-10,30.549999,30.730000,30.309999,30.370001,63532800
2018-01-11,30.660000,30.690001,30.450001,30.660000,59318300
2018-01-12,31.190001,31.200001,30.770000,30.879999,66371600


In [7]:
# we need only average price and volume sum from this dataset
print(yh_df.Close.mean())
print(yh_df.Volume.sum())

31.410819632108094
4608062700


In [8]:
def get_all_data(symbol):
    symbol_data = get_nasdaq_data(symbol)
    if symbol_data:
        symbol_data["symbol"] = symbol
        yahoo_df = get_data_yahoo(symbol, quarter_start, quearter_end)
        if yahoo_df is not None:
            symbol_data["avg_price_for_quarter"] = yahoo_df.Close.mean()
            symbol_data["total_volume_for_quarter"] = yahoo_df.Volume.sum()
        return symbol_data

In [9]:
start = datetime.now()
# now we can pull all the data from nasdaq into the data list
data = []
quarter_start, quearter_end = get_the_quarter_days()
for symbol in symbols[:10]:
    symbol_data = get_all_data(symbol)
    if symbol_data:
        data.append(symbol_data)
df = pd.DataFrame.from_records(data)
print(datetime.now() - start, ", so pulling data in a simple cycle takes too long")
df[:10]

0:00:20.006456 , so pulling data in a simple cycle takes too long


,avg_price_for_quarter,decreased_holders,decreased_shares,held_holders,held_shares,increased_holders,increased_shares,institutional_ownership,new_holders,new_shares,price,sold_holders,sold_shares,symbol,total_shares,total_volume_for_quarter
0,2.385410,26.0,2108211.0,16.0,3.2816e+07,22.0,5332613.0,82.56%,9.0,1417226.0,$1.5,10.0,82250.0,DXLG,49.0,6248900
1,253.711967,338.0,7909378.0,106.0,6.11359e+07,289.0,4740124.0,90.08%,84.0,733226.0,$268.47,56.0,1126100.0,ORLY,82.0,56493800
2,1.332934,5.0,163494.0,0.0,(95211),2.0,18277.0,00.86%,2.0,18277.0,$1.13,2.0,153032.0,DELT,10.0,61542900
3,10.384098,38.0,616770.0,29.0,1.9451e+07,58.0,2097822.0,81.64%,14.0,1104735.0,$12.27,3.0,3524.0,DGII,27.0,6683300
4,41.438360,3.0,22726.0,2.0,410120,5.0,15306.0,04.54%,3.0,15014.0,$34.27,1.0,21.0,CYAD,10.0,242100
5,25.381639,0.0,0.0,1.0,8611,1.0,27.0,00.24%,0.0,0.0,$25.36,0.0,0.0,OXLCO,4.0,142500
6,33.324754,20.0,213691.0,12.0,3.44936e+06,34.0,151383.0,48.69%,8.0,37140.0,$30.78,4.0,27546.0,BWFG,8.0,676100
7,40.885410,27.0,882470.0,5.0,6.8968e+06,39.0,1451252.0,21.47%,12.0,250993.0,$41.5,6.0,62106.0,OMAB,43.0,5481300
8,86.623771,80.0,1199833.0,30.0,9.54759e+06,94.0,1586649.0,98.66%,30.0,259086.0,$83.75,22.0,273837.0,ALOG,13.0,9269000
9,70.133442,12.0,61055.0,6.0,374518,14.0,53962.0,33.76%,5.0,6313.0,$70,0.0,0.0,PYZ,1.0,331200


In [10]:
def save_data_in_parallel(symbols):
    
    num_worker_threads = 20
    input_queue = queue.Queue()
    responses_queue = queue.Queue()
    
    def worker():
        while True:
            symbol = input_queue.get()
            if symbol is None:
                break
                
            data = get_all_data(symbol)
            if data:
                responses_queue.put(data)
                
            input_queue.task_done()

    threads = []
    for i in range(num_worker_threads):
        t = threading.Thread(target=worker)
        t.start()
        threads.append(t)

    for symbol in symbols:
        input_queue.put(symbol)

    # block until all tasks are done
    input_queue.join()

    # stop workers
    for i in range(num_worker_threads):
        input_queue.put(None)
    
    # join threads
    for t in threads:
        t.join()
    
    data = []
    while True:
        try:
            result = responses_queue.get_nowait()
        except queue.Empty:
            break
        else:
            data.append(result)
    return pd.DataFrame.from_records(data)
        
df = save_data_in_parallel(symbols)
df[:10]

Unexpected response for FVE
MRDNW {'chart': {'result': None, 'error': {'description': 'No data found, symbol may be delisted', 'code': 'Not Found'}}}
Unexpected response for MPACW
Unexpected response for KAACW
Unexpected response for NHLDW
Unexpected response for EACQU
MEET {'chart': {'result': None, 'error': {'description': "Data doesn't exist for startDate = 1514816760, endDate = 1522520760", 'code': 'Bad Request'}}}
Unexpected response for INSY
Unexpected response for STLRU
Unexpected response for VEACW
Unexpected response for NXTDW
IVFGC {'chart': {'result': None, 'error': {'description': "Data doesn't exist for startDate = 1514816760, endDate = 1522520760", 'code': 'Bad Request'}}}
Unexpected response for GAINM
ITEK {'chart': {'result': None, 'error': {'description': 'No data found, symbol may be delisted', 'code': 'Not Found'}}}
Unexpected response for GLPI
Unexpected response for WYIG
DSGX {'chart': {'result': None, 'error': {'description': 'No data found, symbol may be delisted

Unexpected response for SNOAW
Unexpected response for MDVXW
Unexpected response for VKTXW
Unexpected response for CLRBZ
WYIGW {'chart': {'result': None, 'error': {'description': 'No data found, symbol may be delisted', 'code': 'Not Found'}}}
CUBN {'chart': {'result': None, 'error': {'description': 'No data found, symbol may be delisted', 'code': 'Not Found'}}}
Unexpected response for ONSIW
Unexpected response for SHIPW
Unexpected response for RNVA
Unexpected response for TICCL
Unexpected response for CAC
Unexpected response for AQMS
Unexpected response for GTYHW
INOD {'chart': {'result': None, 'error': {'description': 'No data found, symbol may be delisted', 'code': 'Not Found'}}}
Unexpected response for CYHHZ
Unexpected response for CATYW
Unexpected response for BHACW
Unexpected response for TLT
Unexpected response for CLRBW
Unexpected response for HUNTW
Unexpected response for ZIONW
PPHM {'chart': {'result': None, 'error': {'description': 'No data found, symbol may be delisted', 'cod

,avg_price_for_quarter,decreased_holders,decreased_shares,held_holders,held_shares,increased_holders,increased_shares,institutional_ownership,new_holders,new_shares,price,sold_holders,sold_shares,symbol,total_shares,total_volume_for_quarter
0,25.381639,0.0,0.0,1.0,8611,1.0,27.0,00.24%,0.0,0.0,$25.36,0.0,0.0,OXLCO,4.0,142500.0
1,1.332934,5.0,163494.0,0.0,(95211),2.0,18277.0,00.86%,2.0,18277.0,$1.12,2.0,153032.0,DELT,10.0,61542900.0
2,25.572771,0.0,0.0,0.0,0,0.0,0.0,0%,0.0,0.0,$25.31,0.0,0.0,HBHCL,6.0,575900.0
3,70.133442,12.0,61055.0,6.0,374518,14.0,53962.0,33.76%,5.0,6313.0,$70,0.0,0.0,PYZ,1.0,331200.0
4,20.222787,7.0,18000.0,6.0,552595,20.0,342219.0,04.86%,8.0,190884.0,$25.1,4.0,9315.0,COOL,19.0,7940500.0
5,41.438360,3.0,22726.0,2.0,410120,5.0,15306.0,04.54%,3.0,15014.0,$34.27,1.0,21.0,CYAD,10.0,242100.0
6,2.385410,26.0,2108211.0,16.0,3.2816e+07,22.0,5332613.0,82.56%,9.0,1417226.0,$1.5,10.0,82250.0,DXLG,49.0,6248900.0
7,33.324754,20.0,213691.0,12.0,3.44936e+06,34.0,151383.0,48.69%,8.0,37140.0,$30.78,4.0,27546.0,BWFG,8.0,676100.0
8,NaN,23.0,622977.0,25.0,9.47388e+06,15.0,421427.0,20.81%,7.0,203654.0,$1.2,5.0,193923.0,FVE,51.0,NaN
9,10.384098,38.0,616770.0,29.0,1.9451e+07,58.0,2097822.0,81.64%,14.0,1104735.0,$12.27,3.0,3524.0,DGII,27.0,6683300.0


In [11]:
def draw_table(sub_df, headers, fields):
    """
    Makes an html table
    """
    s = "<table><tr><th>" + "</th><th>".join(headers) + "</th></tr>"
    for r in sub_df.itertuples():
        s += "<tr><td>" + "</td><td>".join(str(getattr(r, f)) for f in fields) + "</td></tr>"
    s += "</table>"
    display(HTML(s))
    
draw_table(df[:10], ("Stock symbol", "It's price"), ("symbol", "price"))

Stock symbol,It's price
OXLCO,$25.36
DELT,$1.12
HBHCL,$25.31
PYZ,$70
COOL,$25.1
CYAD,$34.27
DXLG,$1.5
BWFG,$30.78
FVE,$1.2
DGII,$12.27


In [12]:
df["ratio"] = (df.increased_holders - df.decreased_holders) / df.held_holders
df.sort_values("ratio", ascending=False, inplace=True)
display(HTML("<h3>Top 100 stocks by Active Bullish Activity By Number of Funds((Increased – Decreased)/Held)</h3>"))

draw_table(
    df[:100], 
    ("Stock", "Share price", "Inst.Ownership", "Increased", "Decreased", "Held", "Ratio"),
    ("symbol", "price", "institutional_ownership", "increased_holders", "decreased_holders", "held_holders", "ratio")
)

Stock,Share price,Inst.Ownership,Increased,Decreased,Held,Ratio
YECO,$3.5,01.27%,4.0,0.0,0.0,inf
PAVMW,$0.39,0%,2.0,0.0,0.0,inf
HLG,$77.73,00.01%,7.0,1.0,0.0,inf
FTFT,$2.1,00.12%,5.0,3.0,0.0,inf
UDBI,$31.4199,00.11%,2.0,0.0,0.0,inf
NFEC,$2.179,00.55%,4.0,3.0,0.0,inf
JSMD,$41.69,59.17%,9.0,3.0,0.0,inf
SWIN,$30.99,42.90%,6.0,3.0,0.0,inf
OPHC,$4.84,00.45%,2.0,1.0,0.0,inf
XGTIW,$3.2425,0%,1.0,0.0,0.0,inf


In [13]:
df["ratio"] = df.increased_shares / df.decreased_shares
df.sort_values("ratio", ascending=False, inplace=True)
display(HTML("<h3>Top 100 stocks by Active Bullish Activity By Shares (Increased / Decreased)</h3>"))

draw_table(
    df[:100], 
    ("Stock", "Share price", "Inst.Ownership", "Increased", "Decreased", "Held", "Ratio"),
    ("symbol", "price", "institutional_ownership", "increased_shares", "decreased_shares", "held_shares", "ratio")
)

Stock,Share price,Inst.Ownership,Increased,Decreased,Held,Ratio
YECO,$3.5,01.27%,30475.0,0.0,0.0,inf
BOSC,$2.2863,00.88%,19705.0,0.0,10000.0,inf
JSM,$23.0473,00.14%,1.0,0.0,15403.0,inf
BNTCW,$0.24,0%,459.0,0.0,140578.0,inf
HIHO,$4.935,04.89%,35121.0,0.0,150686.0,inf
CJJD,$1.51,00.93%,61756.0,0.0,173500.0,inf
LEXEB,$39.57,00.47%,274.0,0.0,12985.0,inf
SNOAW,$0.3986,0%,15066.0,0.0,5702.0,inf
BPFHP,$25.36,06.85%,10716.0,0.0,126291.0,inf
USATP,$26.15,12.69%,114.0,0.0,56350.0,inf


In [14]:
df["ratio"] = df.new_holders / df.sold_holders
df.sort_values("ratio", ascending=False, inplace=True)
display(HTML("<h3>Top 100 stocks by New Bullish Activity By Funds (New / Sold Out)</h3>"))

draw_table(
    df[:100], 
    ("Stock", "Share price", "Inst.Ownership", "New", "Sold out", "Ratio"),
    ("symbol", "price", "institutional_ownership", "new_holders", "sold_holders", "ratio")
)

Stock,Share price,Inst.Ownership,New,Sold out,Ratio
YECO,$3.5,01.27%,4.0,0.0,inf
ISHG,$84.13,25.41%,5.0,0.0,inf
WHLRP,$16.49,45.59%,2.0,0.0,inf
ACST,$0.6506,06.88%,5.0,0.0,inf
PDEX,$6.8843,06.30%,3.0,0.0,inf
VSDA,$27.66,19.00%,4.0,0.0,inf
DDBI,$30.245,89.04%,2.0,0.0,inf
LSXMB,$44.1,00.92%,3.0,0.0,inf
DWAQ,$109.9,17.82%,3.0,0.0,inf
CEMI,$8.4001,34.43%,9.0,0.0,inf


In [15]:
df["ratio"] = df.new_shares / df.sold_shares
df.sort_values("ratio", ascending=False, inplace=True)
display(HTML("<h3>Top 100 stocks by New Bullish Activity By Shares (New / Sold Out)</h3>"))

draw_table(
    df[:100], 
    ("Stock", "Share price", "Inst.Ownership", "New", "Sold out", "Ratio"),
    ("symbol", "price", "institutional_ownership", "new_shares", "sold_shares", "ratio")
)

Stock,Share price,Inst.Ownership,New,Sold out,Ratio
YECO,$3.5,01.27%,30475.0,0.0,inf
PAVMW,$0.39,0%,3949.0,0.0,inf
LTRPB,$12.95,00.58%,30.0,0.0,inf
EVK,$3.995,00.52%,33678.0,0.0,inf
UNAM,$7.55,24.22%,19.0,0.0,inf
EMCG,$25.28,51.62%,13090.0,0.0,inf
SNGXW,$0.4995,0%,200.0,0.0,inf
PNRG,$69.7095,02.42%,14531.0,0.0,inf
UONE,$1.85,09.53%,21041.0,0.0,inf
CHNR,$1.986,00.18%,33239.0,0.0,inf


In [16]:
df["incoming_flow"] = (df.increased_shares + df.new_shares) * df.avg_price_for_quarter
df["outgoing_flow"] = (df.decreased_shares + df.sold_shares) * df.avg_price_for_quarter
df["net_flow"] = df.incoming_flow - df.outgoing_flow
df["ratio"] = df.net_flow / df.total_volume_for_quarter * df.avg_price_for_quarter

df.sort_values("ratio", ascending=False, inplace=True)
display(HTML("<h3>Top 100 stocks by New Bullish Activity By Shares (New / Sold Out)</h3>"))

draw_table(
    df[:100], 
    ("Stock", "Share price", "Inst.Ownership", "Incoming flow", "Outgoing flow",
     "Avg Price", "Total Volume", "Bullish Ratio"),
    ("symbol", "price", "institutional_ownership", "incoming_flow", "outgoing_flow", 
     "avg_price_for_quarter", "total_volume_for_quarter", "ratio")
)

Stock,Share price,Inst.Ownership,Incoming flow,Outgoing flow,Avg Price,Total Volume,Bullish Ratio
USATP,$26.15,12.69%,5289.600173950195,0.0,23.200000762939453,0.0,inf
ISRL,$92.8278,08.74%,23050852.322628085,217006.59177486232,105.13885260409997,31500.0,76213.47113272808
LOXO,$133.5,417.95%,10884586587.504025,429907396.8871948,106.36327887363122,23662900.0,46993.1393990562
GOOGL,$1106,80.36%,16709910994.675415,11957331366.208704,1108.8877123222976,136344900.0,38652.543306276995
UHAL,$343.37,36.74%,753033727.0905097,347610274.07595825,355.4693458432057,4524200.0,31854.38522447642
HWBK,$21.65,22.39%,10178413.940702438,948685.2971096039,20.448888778686523,8500.0,22204.434645916128
AMZN,$1606.025,60.34%,22835418330.703022,19480671723.42929,1430.1942598936987,364555300.0,13161.07416658255
LBTYB,$30.55,00.40%,892535.8459835052,382798.1939907074,32.996999740600586,1300.0,12938.317823523616
CSGP,$392.99,99.28%,1574599982.3302042,1190459124.3550544,344.7181956807121,14211800.0,9317.633477000405
FSV,$70.38,66.71%,334063537.9435893,69506461.55375259,69.0996717859487,2396400.0,7628.4456464646455
